<a href="https://colab.research.google.com/github/leinaxd/Tps/blob/master/a3/a3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS224N Assignment 3: Dependecy Parsing (8 + 44 Points)

En esta tarea, construirás un *Neural Dependency Parser* utilizando *PyTorch*.
En la parte 1, aprenderás un par de técnicas utilizadas en redes neuronales (*Adam optimization* y *Dropout*). En la parte 2, implementarás y entrenarás un *Dependency Parser* utilizando lo visto en la parte 1, antes de analizar unas cuantas dependencias erroneas.

## Parte 1, Machine Learning & Neural Networks (8 points)

### a. Adam Optimizer (4 points)

Recordando la regla de actualización por **Stochastic Gradient Descent**

$\theta := \theta - \alpha \nabla_\theta\ J_{minibatch}(\theta)$

Donde $\theta$ es un vector con todos los parámetros del modelo y $\alpha$ la tasa de aprendizaje.

**i.** (2 points) El optimizador **Adam** utiliza la idea de **momentum** que consiste en **trackear** a $\mathbf{m}$, un promedio desenrollado de los **gradientes**.

$ \begin{align}
\mathbf{m} &:= \beta_1 \mathbf{m} + (1-\beta_1)\nabla_\theta J_{minibatch}(\theta)\\
\theta&:= \theta -\alpha \ \mathbf{m}
\end{align}$

Donde $\beta_1\in[0,1]$ es un hiperparámetro (suele ser 0.9). Explicar brevemente entre 2-4 oraciones (sin demostrar matemáticamente, solo la intuición) cómo utilizando $\mathbf{m}$ puntos evita que las actualizaciones varíen demasiado y por qué esta baja varianza podría ser útil en el apdendizaje general.

<font color='green'>

El minibatch de $\mathbf{m}$ muestras ayuda a encontrar una buena dirección de movimiento antes de desplazarse en dicha dirección.
* Con un $\mathbf{m}$ chico, la dirección cambia tanto que habrá que elegir $\alpha$ chico también y el aprendizaje será lento.
* Con un $\mathbf{m}$ grande, la dirección tiene poca variabilidad, será estable y se podrá recorrer mayor distancia en el espacio de parámetros. El aprendizaje será rápido e incluso permite hacer un line search para encontrar el $\alpha$ óptimo.

</font>

**ii.** (2 points) **Adam** extiende la idea de **momentum** al incluir **tasas de aprendizaje adaptativas**, esto se logra **trackeando** a $\mathbf{v}$, que es un promedio desenrollado en el tiempo de las magnitudes de los gradientes.

$\begin{align}
\mathbf{m} &:= \beta_1 \mathbf{m} + (1-\beta_1)\nabla_\theta J_{minibatch}(\theta)\\
\mathbf{v} &:=\beta_2 \mathbf{v} + (1-\beta_2)(\nabla_\theta J_{minibatch}(\theta)\ \odot\ \nabla_\theta J_{minibatch}(\theta) )\\
\theta&:= \theta -\alpha \ \mathbf{m} \  \oslash\ \sqrt{\mathbf{v}}
\end{align}$

Donde $\odot$ y $\oslash$ representan la multiplicación y división elemento a elemento. $\beta_2\in[0,1]$ (normalmente seteado a 0.99).

Debido a la división de la actualización por $\sqrt{\mathbf{v}}$, Cuál de los parámetros del modelo obtendrán mayores actualizaciones? Por qué esto ayudará al aprendizaje?

<font color = "green">

* Los elementos más chicos de $\mathbf{v}$ serán los que más afecten a los parámetros.<br>
 Como $\mathbf{v}$ decae exponencialmente, entonces las direcciones más lejanas u olvidadas tendrán mayor prioridad.<br>
 El efecto que tiene $\mathbf{v}$ sobre los parámetros es sobre la magnitud del gradiente.
* Las componentes de $\mathbf{m}$ más grandes, representan las direcciones prioritarias a recorrer.
* El momento de 1er orden prioriza la dirección del gradiente (acelerando en las rectas) y el momento de 2do orden prioriza la amplitud de las direcciones no transitadas (acelerando en las curvas).

</font>

### b. Dropout (4 points)
Dropout es una técnica de **Regularización**. Durante el entrenamiento, se pone a **cero** un conjunto aleatorio de unidades en la capa oculta
 $\mathbf{h}$ basados en $p_{drop}$ (descartando diferentes unidades en cada *minibatch* ).
 Luego se multiplica cada $\mathbf{h}$ por una constante $\gamma$.

$$h_{drop} = \gamma \ \mathbf{d} \odot \ \mathbf{h}$$

Donde $\mathbf{d}\in\{0,1\}^{D_h}$ ($D_h$ es el tamaño de $\mathbf{h}$) es una **máscara** y
 **vector** donde cada entrada vale $0$ con probabilidad $p_{drop}$ y 1 en otro caso.
  $\gamma$ se elije de modo que el valor esperado de $\mathbf{h_{drop}}$ sea $\mathbf{h}$.

$$\mathbb{E}_{p_{drop}}[\mathbf{h_{drop}}]_i = h_i \hspace{20mm} \forall i \in \{1\ldots  D_h\}$$

**i.** (2 points) A qué debe ser igual $\gamma$ en términos de $p_{drop}$? Justificar brevemente o mostrar tu derivación matemática utilizando las ecuaciones dadas arriba.

<font color = "green">

$\begin{align}
 E_{p_{drop}}[\mathbf{h_{drop}}]
 &= \gamma E_{p_{drop}}[\  \mathbf{d} \odot \mathbf{h}] &= \mathbf{h} \\
 &= \gamma \mathbf{h}\ \odot\ E_{p_{drop}}[\  \mathbf{d}\ ]&= \mathbf{h} \\
 &= \gamma \mathbf{h}\ \odot\
 (1-p_{drop})&= \mathbf{h} \\
 &\Rightarrow \boxed{\gamma = \frac{1}{(1-p_{drop})}}
 \end{align}
$
</font>

**ii.** (2 points) Por qué debe aplicarse **Dropout** al entrenamiento? Por qué **No** debe aplicarse Dropout en la evaluación?
·Pista: Podría ayudarte, leer el paper.

<font color = "green">
Es un método de regulación, es decir mejora el error de generalización a costa de empeorar el error de entrenamiento.
No tiene sentido empeorar el error en la evaluación.
</font>

1. Kingma and Ba, 2015. https://arxiv.org/pdf/1412.6980.pdf
2. La actualización **Adam** utiliza varios trucos que son menos importantes, los cuales no nos preocuparemos. Para aprender más de ellos, puedes acceder a: https://github.com/cs231n/cs231n.github.io/blob/master/neural-networks-3.md
3. Srivastava et al., 2014, https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf

## Parte 2, Neural Transition-Based Dependency Parsing (44 points)
En esta sección implementarás un *Dependency Parser* neuronal basado en maximizar la métrica UAS (Unlabeled Attachment Score).

Utilizaremos PyTorch 1.7 con la opción *CUDA*  puesta en *None*, y el paquete *tqdm* que produce visualizaciones de *progress bar* durante el aprendizaje.

Un **Dependency Parser** analiza la estructura sintáctica de la oración, estableciendo relaciones entre *palabras cabeceras* y palabras que *modifican* dichas *cabeceras*. Hay diferentes tipos de analizadores de dependencias, incluyendo los *Transition-based parser*, *graph based parser* y *features-based parser*. Tu implementación será basada en transiciones, construído incrementalmente. En cada paso, se mantiene un *partial parser* representado a continuación.

* Una pila (stack) de palabras que están siendo procesadas actualmente.
* Un registro (buffer) de palabras a procesar
* Una lista de dependencias predichas por el analizador.

Inicialmente, el **stack** solamente contiene la raíz **ROOT**, la lista de dependencias está vacía y el **Buffer** contiene todas las palabras de la oración en orden.<br>
En cada paso, el analizador aplica una **transición** al *partial parse* hasta que el *buffer* esté vacío y el tamaño del *stack* sea 1. Las siguientes *transiciones* podrán aplicar:

* **SHIFT** Elimina la primera palabra del **Buffer** y lo apila al **stack**
* **LEFT-ARC** Marca el segundo (a la añadida más recientemente) item sobre el **stack** como dependiente del primer ítem y elimina el segundo item del **stack**, añadiendo a la lista de dependencias la dependecia de la $primera\_palabra \rightarrow segunda\_palabra$.
* **RIGHT-ARC** Marca el primer (de la añadida más recientemente) item sobre el **stack** como dependiente del segundo item y elimina el primer item del **stack**, añadiendo a la lista de dependencias, la dependencia desde la $segunda\_palabra \rightarrow primera\_palabra$.

![transitions](https://github.com/leinaxd/Tps/raw/master/a3/imgs/transitions.jpg)

En cada paso, el **parser** decidirá entre las 3 transiciones utilizando un **Clasificador Neuronal**.

a. (4 points) Atravezar la secuencia de transiciones requeridas para parsear la oración *"I parsed this sentence correctly"*. El árbol de dependencias es el mostrado debajo. <br>
En cada paso, dar la configuración del **stack**, **buffer**, qué transición fue aplicada en este paso y qué nueva dependencia fue añadida (si la hubo). Los primeros 3 pasos se ofrecen como ejemplo.

![oracion parseada](https://github.com/leinaxd/Tps/raw/master/a3/imgs/2a-Parsed-sentence.jpg)

<font color = "green">

| Stack | Buffer | New Dependency |Transition |
|---------------|----------|----------------|-----------|
|[ROOT]| [I, parsed, this, sentence, correctly] | · |Initial Configuration|
|[ROOT, I]| [parsed, this, sentence, correctly] | · |SHIFT|
|[ROOT, I, parsed]| [this, sentence, correctly] | · |SHIFT|
|[ROOT, parsed]| [this, sentence, correctly] | parsed -> I |LEFT-ARC|
| - | - | - | - |
|[ROOT, parsed, this]| [sentence, correctly] | - |SHIFT|
|[ROOT, parsed, this, sentence]| [correctly] | - |SHIFT|
|[ROOT, parsed, sentence]| [correctly] | sentence -> this |LEFT-ARC|
|[ROOT, parsed]| [correctly] | parsed -> sentence |RIGHT-ARC|
|[ROOT, parsed, correctly]| [] | - |SHIFT|
|[ROOT, parsed]| [] | parsed -> correctly |RIGHT-ARC|
|[ROOT]| [] | ROOT -> parsed |RIGHT-ARC|

</font>

b. (2 points) En cuantos pasos será parseada una oración que contiene $n$ palabras? (en términos de $n$) Explicar en 1-2 oraciones por qué.

<font color = "green">

* Se necesitan $n$ shifts (llevar cada palabra del *Buffer* al *stack*)
* Si solamente se aplican RIGHT-ARC,
 reducir $n$ símbolos lleva $n+1$ pasos (más el símbolo ROOT)

 Por ende se requieren $\boxed{2n+1}$ pasos parsear una oración de $n$ palabras.

</font>

c. (6 points) Implementar las funciones **__init__** y **parse_step** de la clase **PartialParse** dentro de **parser_transition.py**. Esta implementa el mecanismo de transición que utilizará tu parser. Puedes ejecutar tests básicos (no-exhaustivos) ejecutando: <br>
` python parser_transitions.py part_c `


In [19]:
!python parser_transitions.py part_c

SHIFT test passed!
LEFT-ARC test passed!
RIGHT-ARC test passed!
parse test passed!


d. (8 points) Nuestra red predicirá qué transición deberia ser la siguiente en aplicarse a ese parseo parcial. Podríamos utilizarlo para parsear una oración aplicando transiciones hasta completar el parseo.
Sin embargo, las redes neuronales funcionan mucho más eficientes cuando toma predicciones sobre lotes (batches) de datos en ese instante (ej. Predecir la siguiente transición para diferentes parseos parciales simultáneamente). Podemos parsear oraciones en minilotes (minibatches) con el siguiente algoritmo.

![algoritmo](https://github.com/leinaxd/Tps/raw/master/a3/imgs/2d-Algorithm.jpg)

* Implementar este algoritmo en la función **minibatch_parse** en **parser_transitions.py**.

·Puedes ejecutar diferentes tests básicos ejecutando
`python parser_transitions.py part_d`<br>
·Obs. Necesitarás que **minibatch_parse** esté implementada correctamente para evaluar el modelo de la parte *(e)*. Sin embargo, no necesitas entrenar el modelo, por lo que podrás completar la mayor parte del (e) incluso si *minibatch_parse* no está implementada aún.

In [36]:
!python parser_transitions.py part_d

minibatch_parse test passed!


e. (12 points) Ahora entrenaremos el modelo neuronal para predecir la siguiente transición, dado el estado del **stack**, el **buffer** y las **dependencias**.<br>

Primero, el modelo extrae un **vector de features** representando el estado actual. Utilizaremos el **set de features** presentado en el paper original: *"A Fast and Accurate Dependency Parser using Neural Networks"¬Chen and Manning, 2014. https://nlp.stanford.edu/pubs/emnlp2014-depparser.pdf*.<br>
La función que extrae estos features ha sido implementada para ti en *utils/parser_utils.py*.<br>
Este vector de Features consiste en una lista de símbolos (tokens) (como la última palabra del stack, primera palabra del buffer, *dependiente* de la anteúltima palabra del stack si este existe, etc.). Estos pueden estar representados como una lista de enteros $\mathbf{w}=[w_1, w_2, \ldots, w_m]$ donde $m$ es la cantidad de features y cada $0\le w_i \lt |V|$ es el índice del símbolo en el vocabulario. Luego nuestra red observa los embeddings para cada palabra y los **concatena** en un solo vector de entrada.

$\mathbf{x} = [\mathbf{E}_{w_1}, \ldots, \mathbf{E}_{w_m}]\in \mathbb{R}^{dm}$

Donde $\mathbf{E}\in \mathbb{R}^{|V|\times d}$ es la matriz de
 codificación (embedding) donde cada **fila** $\mathbf{E}_w$ es el vector
  para determinada palabra $w$.

Luego calculamos la predicción según:

$\begin{align}
\mathbf{h} &= ReLu(\mathbf{xW} + \mathbf{b_1})\\
\mathbf{l} &= \mathbf{hU}+\mathbf{b_2}\\
\mathbf{\hat y} &= softmax(\mathbf{l})
\end{align}$

Donde $\mathbf{h}$ se refiere a la capa oculta, $\mathbf{l}$ se refiere a los logits, $\mathbf{\hat y}$ se refiere a las predicciones y $ReLu = max(z,0)$.
Entrenaremos el modelo minimizando la entropía cruzada:

$J(\theta) = CE(y,\hat y) = - \sum_{i=1}^3 y_i\ log(\hat y_i)$

Para calcular el costo (loss) del set de entrenamiento, **promediaremos** esta $J(\theta)$ sobre todos los ejemplos.
Utilizaremos el puntaje (score) **UAS** como métrica de evaluación. UAS significa *Unlabeled Attachment Score*, que es calculado como, la **relación** entre las dependencias **correctamente predichas** y las dependencias **totales** (incluyendo aquellas que nuestro modelo no predice).

En **parser_model.py** encontraras un esqueleto de código para implementar esta simple red neuronal utilizando PyTorch. Complete las funciones **__init__**, **embedding_lookup** y **foward**. Luego complete las funciones **train_for_epoch** y **train** dentro del archivo **run.py**. Finalmente ejecute `python run.py` para entrenar el modelo y obtener las predicciones sobre el **DataSet de testeo** de **Penn Treebank** (anotado con dependencias universales)

Observaciones:
* Para esta tarea, se te pide implementar una capa lineal y una capa oculta (embedding). Por favor, **No utilizar** los módulos **torch.nn.Linear** o **torch.nn.Embedding**, caso contrario se te deducirán puntos.
* Por favor, sigue los requerimientos en el **TODO**. Ej. si hay una recomendación explícita sobre los nombres de variables.

Pistas:
* Cada palabra que se te pide declarar (self.embed_to_hidden_weight, self.embed_to_hidden_bias, self.hidden_to_logits_weight, self.hidden_to_logits_bias) corresponde a una de las variables de arriba ($\mathbf{W}$, $\mathbf{b_1}$, $\mathbf{U}$, $\mathbf{b_2}$)
* Podría ayudar trabajar en el algoritmo desde el final (empezar por $\hat y$) y tener trackeadas las dimensiones de las matrices y vectores.
* Una vez hayas implementado **embedding_lookup** (e) o **forward** (f) puedes ejecutar `python parser_model.py` con los flags `-e` o `-f` o ambos para ejecutar checkeos de sanidad con cada función. Estos checkeos son bastante básicos y pasarlos no garantizan que tu código esté libre de errores.
* Al debuggear, puedes añadir el flag `python run.py -d`. Esto hará que el código se ejecute sobre un pequeño dataset, por lo que el entrenamiento no tardará mucho. Asegurarse de eliminar el flag al ejecutar el modelo completo una vez finalizado el debugging.
* Al ejecutar en modo debug, deberías obtener un **coste** menor a **0.2** y un **UAS** mayor al **65%** del set de entrenamiento
* Debería tomarte **1 hora** entrenar al modelo sobre el dataset completo. (modo normal)
* En modo normal, deberías obtener un **coste** menor a **0.8** y un **UAS** mayor al **87%** del set de entrenamiento. En comparación el modelo del paper original obtuvo un **UAS** del 92.5. Si lo deseas, puedes afinar los hiperparámetros del modelo (Tamaño de la capa oculta, hiperparámetros del optimizador, cantidad de épocas).

Módulos:
* La mecánica de transiciones está en **parser_transition.py**
* Minibatch Dependency Parsing en **parser_transitions.py**
* Neural Dependency Parsing en **parser_model.py**
* Funciones de entrenamiento en **run.py**
* Registrar escritamente, el mejor **UAS** obtenido en el **set de desarrollo** y en el **set de entrenamiento**

In [3]:
!python parser_model.py -e

Traceback (most recent call last):
  File "D:\Estudio\Stanford\NLP with Deep Learning, Winter 2019\Tps\a3\parser_model.py", line 12, in <module>
    import torch
ModuleNotFoundError: No module named 'torch'


(f) (12 points) Queremos analizar un ejemplo de *dependency parsing* y comprender en donde se equivoca. Por ejemplo en la secuencia

![incorrecto](https://github.com/leinaxd/Tps/raw/master/a3/imgs/2f-ejemplo_incorrecto.jpg)

La dependencia sobre la frase *Afghanistan* está mal, debido a que la frase debería modificar *sent* (ya que es enviado a afghanistan) no *troops*. A continuación la frase correcta.

![correcto](https://github.com/leinaxd/Tps/raw/master/a3/imgs/2f-ejemplo_correcto.jpg)

En general habrá 4 tipos de errores:
* **Prepositional Phrase Attachment Error**, En el ejemplo de arriba, la frase *into Afghanistan* es una *Prepositional Phrase*. El *Prepositional Phrase Attachment Error* se da cuando una *Prepositional Phrase* se liga a una *head word* incorrecta (en este ejemplo, *troops* es la cabecera incorrecta y *sent* es la correcta). Otros ejemplos incluyen *with a rock, before midnight and under the carpet*
* **Verb Phrase Attachment Error**, En la oración *Leaving the store unatended, I went outside to watch the prade*, la frase *leaving the store unatended* es un *Verb Phrase*. El error ocurre cuando el *Verb Phrase* está ligado a una *head word* incorrecta (en el ejemplo, la cabecera correcta era *went*.
* **Modifier Attachment Error**, En la oración, *I am extremely short* el adverbio *extremely* es un modificador del adjetivo *short*. El error ocurre cuando el modificador es ligado al *head word* incorrecto. (El correcto era *short*)
* **Coordination Attachment Error**, En la oración *Would you like brown rice or garlic naan?*, las frases *brown rice* y *garlic* son *conjuncts* y la palabra *or* es el *coordinating conjunction*. El segundo *conjunct* (aquí *garlic naan*) debería estar ligado al primer *conjunct* (aquí *brown rice*. El error es cuando el segundo *conjunct* se liga a la *head word* incorrecta. (La correcta era *rice*). Otros *coordinating conjunctions* son *and*, *but* y *so*

Para ejemplos de *Prepositional Phrase* ir a http://www.grammarly.com/blog/prepositional-phrase
Para ejemplos de *Verb Phrase* ir a https://examples.yourdictionary.com/verb-phrase-examples.html

En esta pregunta se presentan 4 oraciones obtenidas del parser. Cada oración tiene 1 tipo de error y solamente hay un ejemplo de cada uno de los 4 tipos.
Para cada oración indicar el tipo de error, la dependencia incorrecta y la dependencia correcta. Mientras que el tipo de error en cada oración es único, las dependencias correctas podrían ser varias. Para demostrarlo, debes escribir:
* **Tipo de error**, *Prepositional Phrase Attachment Error*
* **Incorrect Dependency**, *troops -> Afghanistan*
* **Correct Dependency**, *sent -> Afghanistan*

Observaciones
Existen muchos detalles y convenciones al anotar dependencias. Para aprender más sobre ellas puedes visitar: https://universaldependencies.org (En esta tarea solamente utilizaremos UDv1 https://universaldependencies.org/docsv1/
o las diapositivas cortas de https://people.cs.georgetown.edu/nschneid/p/UD-for-English.pdf.
Obervar que no necesitas conocer todos estos detalles para responder esta pregunta. En cada caso, estamos preguntando sobre la ligadura de frases y debe ser suficiente con ver si están modificando la palabra correcta. En particular, no necesitas mirar las etiquetas de los arcos de dependencia- Alcanza solamente con ver que existe el arco.

1.
![ej1](https://github.com/leinaxd/Tps/raw/master/a3/imgs/2f-ej_i.jpg)
2.
![ej2](https://github.com/leinaxd/Tps/raw/master/a3/imgs/2f-ej_ii.jpg)
3.
![ej3](https://github.com/leinaxd/Tps/raw/master/a3/imgs/2f-ej_iii.jpg)
4.
![ej4](https://github.com/leinaxd/Tps/raw/master/a3/imgs/2f-ej_iv.jpg)

